# Importing Required Libraries

In [1]:
# For data handling and Plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

# For reading websites
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

# For saving the data
from csv import writer

In [2]:
# Website to Scrape- here incase is PROGRAMIZ.COM
website = 'https://www.programiz.com'
request = requests.get(website)
soup = BeautifulSoup(request.content, 'html.parser')

tab_link_container = soup.find_all('a', class_ = 'highlight-course__col')

In [3]:
# creating variable to name the courses and iterate over the courses
all_course_headings = []
all_course_links = []

In [4]:
# All courses offered by the website
for i in tab_link_container:
    all_course_headings.append(i.h3.text)
    all_course_links.append(i.get('href'))

In [5]:
# Viewing the courses offered
all_course_headings

['Python Programming',
 'C Programming',
 'Java Programming',
 'Javascript Programming',
 'C++ Programming',
 'DS & Algorithms',
 'Kotlin Programming',
 'Swift Programming',
 'C# Programming']

In [6]:
# Viewing the Links of the respective Courses
all_course_links

['/python-programming',
 '/c-programming',
 '/java-programming',
 '/javascript',
 '/cpp-programming',
 '/dsa',
 '/kotlin-programming',
 '/swift-programming',
 '/csharp-programming']

In [7]:
# Function to get the scrape html document from any webpage
def get_data(link):
    request = requests.get(link)
    soup = BeautifulSoup(request.content, 'html.parser')
    return soup

# Scrapping the data from the Website

In [8]:
for link_index in range(len(all_course_links)):
    
    #*********************************************************************************************************************
    # Universal Variables for each dataset
    course_description = []
    main_headings = []
    sub_page_content_list = []
    table_data = []
    page_headings = []
    unordered_list_items = []
    ordered_list_items = []
    site_links = []
    
    #*********************************************************************************************************************
    c_link = website + str(all_course_links[link_index])
    site_links.append(c_link)
    course_soup = get_data(c_link)
    right_bar = course_soup.find_all('div', class_ = 'right-bar')
    course_container = course_soup.find_all('div', class_= 'container')
    editor_contents = right_bar[0].find_all('div', class_ = 'editor-contents')
    
    # All links in main page
    for links in right_bar[0].find_all('a'):
        link = str(links.get('href'))
        if 'online-compiler' in link:
            continue
        if 'programming' in link:
            if (website + link) not in site_links:
                site_links.append(website + link)

    
    # Traversing page wise from the site_links
    for links in range(len(site_links)):

        page_soup = get_data(site_links[links])
        main_headings.append(page_soup.title.text)
        try:
            right_bar = page_soup.find_all('div', class_ = 'right-bar')
            course_container = page_soup.find_all('div', class_= 'container')
            editor_contents = page_soup.find_all('div', class_ = 'editor-contents')
            k = 0
            if len(right_bar) == 0:
                right_bar = page_soup.find_all('div', class_ = 'col-sm-12 col-reset')
                if len(right_bar) == 0:
                    right_bar = page_soup.find_all('div', class_ = 'col-sm-12')
                    k = 2
                    editor_contents = page_soup.find_all('div', class_ = 'card card--reference')
                else:
                    editor_contents = page_soup.find_all('div', class_ = 'tabbed-content__right')
        except IndexError:
            pass
        
        # Programm Description
        description_str = ''
        for desc in course_container:
            description = desc.find_all('div', class_ = 'program-description')
            if len(description) != 0:
                description_str += description[0].text.replace('\n', ' ').strip() + '|||'
        course_description.append(description_str[:-3])
        
        # Description and List Items per page
        for page_data in editor_contents:
            page_paragraphs = page_data.find_all('p')
        
        content_per_page = ''
        for i in page_paragraphs:
            content_per_page += i.text + '|||'
        content_per_page = content_per_page[:-3].replace('\n', '')
        sub_page_content_list.append(content_per_page)
            
        prev = ''
        unordered_list_items_per_page = []
        ordered_list_items_per_page = []
        unorder_order_headings_per_page = []
        h2_h3_headings = []
        for i in editor_contents:
            ul_ol_items = ''
            if len(i) != 0:
                ul = i.find_all('ul')
                for li in ul:
                    for sub_li in li:
                        item = sub_li.text.replace('\n', '').replace('\t','')
                        if len(item) > 0:
                            unordered_list_items_per_page.append(item)

                ul_ol_items = ''
                ol = i.find_all('ol')
                for li in ol:
                    for sub_li in li:
                        item = sub_li.text.replace('\n', '').replace('\t','')
                        if len(item) > 0:
                            ordered_list_items_per_page.append(item)

        # Headings in the webpage
        for i in editor_contents[0].find_all('h2'):
           h2_h3_headings.append(i.text.replace('\n','').replace('\t', '').replace('      ', ''))
        for i in editor_contents[0].find_all('h3'):
           h2_h3_headings.append(i.text.replace('\n','').replace('\t', '').replace('      ', ''))
        
        
        ordered_list_items.append('$$$'.join(ordered_list_items_per_page))
        unordered_list_items.append('$$$'.join(unordered_list_items_per_page))
        page_headings.append('$$$'.join(h2_h3_headings))
    
    
        # Tabular Data per page
        tables = page_soup.find_all('table')
        data_str = ''
        if len(tables) > 0:
            data = []
            for table in tables:
                all_rows = table.find_all('tr')
                all_th = table.find_all('th')
                all_th_str = ''
                for th in all_th:
                    all_th_str += th.text + '|||'
                if len(all_th_str[:-3]) > 0:
                    data.append(all_th_str[:-3])
                for row in all_rows:
                    row_list = row.find_all('td')
                    rows = ''
                    for ro in row_list:
                        rows += ro.text + '|||'

                    if len(rows[:-3]) > 0:
                        data.append(rows[:-3])
                    
            data_str = '$$$'.join(data)
        else:
            data_str = None
        table_data.append(data_str)
    
    #*********************************************************************************************************************
    # Writing the file
    with open(all_course_headings[link_index] + '.csv', 'w', encoding='utf8', newline='') as f:
        the_writer = writer(f)
        header = ['Links','Main Heading','Chapter Headings', 'Description', 
                  'Contents', 'Unordered Lists', 'Ordered Lists', 'Table Data']
        the_writer.writerow(header)
        for i in range(len(site_links)):
            row = [site_links[i], main_headings[i], page_headings[i], course_description[i],
                   sub_page_content_list[i], unordered_list_items[i], ordered_list_items[i], table_data[i]]
            the_writer.writerow(row)

    # Reading the file
    file = pd.read_csv(all_course_headings[link_index] + '.csv')
    dataset = pd.DataFrame(file)

In [9]:
# All the datasets we have are: 
dataset_0 = pd.read_csv(all_course_headings[0] + '.csv')
dataset_1 = pd.read_csv(all_course_headings[1] + '.csv')
dataset_2 = pd.read_csv(all_course_headings[2] + '.csv')
dataset_3 = pd.read_csv(all_course_headings[3] + '.csv')
dataset_4 = pd.read_csv(all_course_headings[4] + '.csv')
dataset_5 = pd.read_csv(all_course_headings[5] + '.csv')
dataset_6 = pd.read_csv(all_course_headings[6] + '.csv')
dataset_7 = pd.read_csv(all_course_headings[7] + '.csv')
dataset_8 = pd.read_csv(all_course_headings[8] + '.csv')

In [19]:
print(all_course_headings[0])
print('----------------------------------------------------------------------------------------')
dataset_0 = dataset_0.replace(np.nan, '', regex=True) # removing the NaN values from the dataset
dataset_0

Python Programming
----------------------------------------------------------------------------------------


,Links,Main Heading,Chapter Headings,Description,Contents,Unordered Lists,Ordered Lists,Table Data
0,https://www.programiz.com/python-programming,Learn Python Programming,About Python Programming$$$Why Learn Python?$$...,Python is a powerful general-purpose programmi...,,Free and open-source - You can freely use and ...,,
1,https://www.programiz.com/python-programming/f...,How to Get Started With Python?,Video: Introduction to Python$$$The Easiest Wa...,,"In this tutorial, you will learn to install an...",Run Python using Thonny IDE$$$Install Python S...,Download Thonny IDE.$$$Run the installer to in...,
2,https://www.programiz.com/python-programming/k...,Python Keywords and Identifiers (Variable names),Python Keywords$$$Python Identifiers$$$Rules f...,,"In this tutorial, you will learn about keyword...",Python Keywords$$$Python IdentifiersRules for ...,Identifiers can be a combination of letters in...,False|||await|||else|||import|||pass$$$None|||...
3,https://www.programiz.com/python-programming/s...,"Python Statement, Indentation and Comments",Python Statement$$$Python Indentation$$$Python...,,"In this tutorial, you will learn about Python ...",Python StatementMulti-line statement$$$Python ...,,
4,https://www.programiz.com/python-programming/v...,"Python Variables, Constants and Literals",Video: Python Variables and print()$$$Python V...,,"In this tutorial, you will learn about Python ...",We assigned integer literals into different va...,Constant and variable names should have a comb...,
5,https://www.programiz.com/python-programming/v...,Python Data Types,Data types in Python$$$Python Numbers$$$Python...,,"In this tutorial, you will learn about differe...",Data types in Python$$$Python Numbers$$$Python...,,
6,https://www.programiz.com/python-programming/t...,Python Type Conversion and Type Casting (With ...,Type Conversion$$$Implicit Type Conversion$$$E...,,"In this article, you will learn about the Type...","We add two variables num_int and num_flo, stor...",Implicit Type Conversion$$$Explicit Type Conve...,
7,https://www.programiz.com/python-programming/i...,Python Input Output (I/O) Using input() and pr...,Video: Python Take User Input$$$Python Output ...,,This tutorial focuses on two built-in function...,Python Output Using print() functionOutput for...,,
8,https://www.programiz.com/python-programming/o...,"Python Operators: Arithmetic, Comparison, Logi...",Video: Operators in Python$$$What are operator...,,"In this tutorial, you'll learn everything abou...",What are operators in Python?$$$Arithmetic Ope...,,Operator|||Meaning|||Example$$$+|||Add two ope...
9,https://www.programiz.com/python-programming/n...,Python Namespace and Scope of a Variable,What is Name in Python?$$$What is a Namespace ...,,"In this tutorial, you will learn about namespa...",What is Name in Python?$$$What is a Namespace$...,Scope of the current function which has local ...,


In [20]:
print(all_course_headings[1])
print('----------------------------------------------------------------------------------------')
dataset_1 = dataset_1.replace(np.nan, '', regex=True) # removing the NaN values from the dataset
dataset_1

C Programming
----------------------------------------------------------------------------------------


,Links,Main Heading,Chapter Headings,Description,Contents,Unordered Lists,Ordered Lists,Table Data
0,https://www.programiz.com/c-programming,Learn C Programming,About C Programming$$$Why Learn C Programming?...,C is a powerful general-purpose programming la...,,Procedural Language - Instructions in a C prog...,,
1,https://www.programiz.com/c-programming/c-keyw...,C Keywords and Identifiers,Character set$$$C Keywords$$$C Identifiers$$$A...,,"In this tutorial, you will learn about keyword...",Character set$$$C Keywords$$$C Identifiers$$$R...,A valid identifier can have letters (both uppe...,",|||<|||>|||.|||_$$$(|||)|||;|||$|||:$$$%|||[|..."
2,https://www.programiz.com/c-programming/c-vari...,"C Variables, Constants and Literals",Video: Variables in C Programming$$$Variables$...,,"In this tutorial, you will learn about variabl...",decimal (base 10)$$$octal (base 8)$$$hexadecim...,A variable name can only have letters (both up...,Escape Sequences|||Character$$$\b|||Backspace$...
3,https://www.programiz.com/c-programming/c-data...,C Data Types,Video: Data Types in C Programming$$$Basic typ...,,"In this tutorial, you will learn about basic d...",signed - allows for storage of both positive a...,,Type|||Size (bytes)|||Format Specifier$$$int||...
4,https://www.programiz.com/c-programming/c-inpu...,C Input/Output: printf() and scanf(),Video: Get User Input in C Programming$$$C Out...,,"In this tutorial, you will learn to use scanf(...",All valid C programs must contain the main() f...,,Data Type|||Format Specifier$$$int|||%d$$$char...
5,https://www.programiz.com/c-programming/c-oper...,Operators in C,Video: Arithmetic Operators in C$$$C Arithmeti...,,"In this tutorial, you will learn about differe...",(a == b) && (c > 5) evaluates to 1 because bot...,,Operator|||Meaning of Operator$$$+|||addition ...
6,https://www.programiz.com/c-programming/c-intr...,C Introduction Examples,Examples,,"In this article, you will find a list of simpl...",,Variables and Constants$$$Data Types$$$Input a...,C program to print a sentence$$$C program to p...
7,https://www.programiz.com/c-programming/c-if-e...,C if...else Statement,Video: C if else Statement$$$C if Statement$$$...,,"In this tutorial, you will learn about the if ...","If the test expression is evaluated to true, s...",,
8,https://www.programiz.com/c-programming/c-for-...,C for Loop (With Examples),Video: C while Loop$$$for Loop$$$How for loop ...,,"In this tutorial, you will learn to create for...",The initialization statement is executed only ...,for loop$$$while loop$$$do...while loop$$$i is...,
9,https://www.programiz.com/c-programming/c-do-w...,C while and do...while Loop,Video: C while Loop$$$while loop$$$do...while ...,,"In this tutorial, you will learn to create whi...",The while loop evaluates the testExpression in...,for loop$$$while loop$$$do...while loop$$$When...,


In [23]:
print(all_course_headings[2])
print('----------------------------------------------------------------------------------------')
dataset_2 = dataset_2.replace(np.nan, '', regex=True) # removing the NaN values from the dataset
dataset_2

Java Programming
----------------------------------------------------------------------------------------


,Links,Main Heading,Chapter Headings,Description,Contents,Unordered Lists,Ordered Lists,Table Data
0,https://www.programiz.com/python-programming,Learn Python Programming,About Python Programming$$$Why Learn Python?$$...,Python is a powerful general-purpose programmi...,,Free and open-source - You can freely use and ...,,
1,https://www.programiz.com/python-programming/f...,How to Get Started With Python?,Video: Introduction to Python$$$The Easiest Wa...,,"In this tutorial, you will learn to install an...",Run Python using Thonny IDE$$$Install Python S...,Download Thonny IDE.$$$Run the installer to in...,
2,https://www.programiz.com/python-programming/k...,Python Keywords and Identifiers (Variable names),Python Keywords$$$Python Identifiers$$$Rules f...,,"In this tutorial, you will learn about keyword...",Python Keywords$$$Python IdentifiersRules for ...,Identifiers can be a combination of letters in...,False|||await|||else|||import|||pass$$$None|||...
3,https://www.programiz.com/python-programming/s...,"Python Statement, Indentation and Comments",Python Statement$$$Python Indentation$$$Python...,,"In this tutorial, you will learn about Python ...",Python StatementMulti-line statement$$$Python ...,,
4,https://www.programiz.com/python-programming/v...,"Python Variables, Constants and Literals",Video: Python Variables and print()$$$Python V...,,"In this tutorial, you will learn about Python ...",We assigned integer literals into different va...,Constant and variable names should have a comb...,
5,https://www.programiz.com/python-programming/v...,Python Data Types,Data types in Python$$$Python Numbers$$$Python...,,"In this tutorial, you will learn about differe...",Data types in Python$$$Python Numbers$$$Python...,,
6,https://www.programiz.com/python-programming/t...,Python Type Conversion and Type Casting (With ...,Type Conversion$$$Implicit Type Conversion$$$E...,,"In this article, you will learn about the Type...","We add two variables num_int and num_flo, stor...",Implicit Type Conversion$$$Explicit Type Conve...,
7,https://www.programiz.com/python-programming/i...,Python Input Output (I/O) Using input() and pr...,Video: Python Take User Input$$$Python Output ...,,This tutorial focuses on two built-in function...,Python Output Using print() functionOutput for...,,
8,https://www.programiz.com/python-programming/o...,"Python Operators: Arithmetic, Comparison, Logi...",Video: Operators in Python$$$What are operator...,,"In this tutorial, you'll learn everything abou...",What are operators in Python?$$$Arithmetic Ope...,,Operator|||Meaning|||Example$$$+|||Add two ope...
9,https://www.programiz.com/python-programming/n...,Python Namespace and Scope of a Variable,What is Name in Python?$$$What is a Namespace ...,,"In this tutorial, you will learn about namespa...",What is Name in Python?$$$What is a Namespace$...,Scope of the current function which has local ...,


In [24]:
print(all_course_headings[3])
print('----------------------------------------------------------------------------------------')
dataset_3 = dataset_3.replace(np.nan, '', regex=True) # removing the NaN values from the dataset
dataset_3

Javascript Programming
----------------------------------------------------------------------------------------


,Links,Main Heading,Chapter Headings,Description,Contents,Unordered Lists,Ordered Lists,Table Data
0,https://www.programiz.com/javascript,Learn JavaScript Programming,About JavaScript Programming$$$Why Learn JavaS...,JavaScript is a powerful and flexible programm...,Prototype-based Object-oriented - JavaScript i...,JavaScript is used on both the client-side and...,,


In [25]:
print(all_course_headings[4])
print('----------------------------------------------------------------------------------------')
dataset_4 = dataset_4.replace(np.nan, '', regex=True) # removing the NaN values from the dataset
dataset_4

C++ Programming
----------------------------------------------------------------------------------------


,Links,Main Heading,Chapter Headings,Description,Contents,Unordered Lists,Ordered Lists,Table Data
0,https://www.programiz.com/cpp-programming,Learn C++ Programming,About C++ Programming$$$Why Learn C++?$$$How t...,C++ is a powerful general-purpose programming ...,,Multi-paradigm Language - C++ supports at leas...,,
1,https://www.programiz.com/cpp-programming/vari...,"C++ Variables, Literals and Constants",C++ Variables$$$C++ Literals$$$C++ Constants$$...,,"In this tutorial, we will learn about variable...","A variable name can only have alphabets, numbe...",,Escape Sequences|||Characters$$$\b|||Backspace...
2,https://www.programiz.com/cpp-programming/data...,C++ Data Types,C++ Fundamental Data Types$$$C++ Type Modifier...,,"In this tutorial, we will learn about basic da...",The int keyword is used to indicate integers.$...,signed$$$unsigned$$$short$$$long,Data Type|||Meaning|||Size (in Bytes)$$$int|||...
3,https://www.programiz.com/cpp-programming/inpu...,C++ Basic Input/Output,C++ Output$$$C++ Input$$$C++ Taking Multiple I...,,"In this tutorial, we will learn to use the cin...",We first include the iostream header file that...,,
4,https://www.programiz.com/cpp-programming/type...,C++ Type Conversion,Implicit Type Conversion$$$C++ Explicit Conver...,,"In this tutorial, we will learn about the basi...",static_cast$$$dynamic_cast$$$const_cast$$$rein...,Implicit Conversion$$$Explicit Conversion (als...,
5,https://www.programiz.com/cpp-programming/oper...,C++ Operators,1. C++ Arithmetic Operators$$$2. C++ Assignmen...,,"In this tutorial, we will learn about the diff...",++ increases the value of the operand by 1$$$-...,Arithmetic Operators$$$Assignment Operators$$$...,Operator|||Operation$$$+|||Addition$$$-|||Subt...
6,https://www.programiz.com/cpp-programming/comm...,C++ Comments,Single Line Comments$$$Multi-line comments$$$U...,,"In this tutorial, we will learn about C++ comm...",// declaring a variable$$$// initializing the ...,,
7,https://www.programiz.com/cpp-programming/if-else,C++ If...else (With Examples),C++ if Statement$$$C++ if...else$$$C++ if...el...,,"In this tutorial, we will learn about the if.....","if the percentage is above 90, assign grade A$...",if statement$$$if...else statement$$$if...else...,
8,https://www.programiz.com/cpp-programming/for-...,C++ for Loop (With Examples),C++ for loop$$$Flowchart of for Loop in C++$$$...,,"In this tutorial, we will learn about the C++ ...",for loop$$$while loop$$$do...while loop$$$init...,,Iteration|||Variable|||i <= 5|||Action$$$1st||...
9,https://www.programiz.com/cpp-programming/do-w...,C++ while and do...while Loop (With Examples),C++ while Loop$$$C++ do...while Loop$$$Infinit...,,"In this tutorial, we will learn the use of whi...",A while loop evaluates the condition$$$If the ...,for loop$$$while loop$$$do...while loop,Iteration|||Variable|||i <= 5|||Action$$$1st||...


In [26]:
print(all_course_headings[5])
print('----------------------------------------------------------------------------------------')
dataset_5 = dataset_5.replace(np.nan, '', regex=True) # removing the NaN values from the dataset
dataset_5

DS & Algorithms
----------------------------------------------------------------------------------------


,Links,Main Heading,Chapter Headings,Description,Contents,Unordered Lists,Ordered Lists,Table Data
0,https://www.programiz.com/dsa,Learn Data Structures and Algorithms,Why Learn DSA?$$$How you can learn data struct...,A computer program is a collection of instruct...,Programiz offers a complete series of easy to ...,Write optimized and scalable code - Once you h...,,
1,https://www.programiz.com/dsa/dynamic-programming,Dynamic Programming,Dynamic Programming Example$$$How Dynamic Prog...,,"In this tutorial, you will learn what dynamic ...",Introduction$$$Dynamic Programming Example$$$H...,The first term is 0.$$$The second term is 1.$$...,


In [27]:
print(all_course_headings[6])
print('----------------------------------------------------------------------------------------')
dataset_6 = dataset_6.replace(np.nan, '', regex=True) # removing the NaN values from the dataset
dataset_6

Kotlin Programming
----------------------------------------------------------------------------------------


,Links,Main Heading,Chapter Headings,Description,Contents,Unordered Lists,Ordered Lists,Table Data
0,https://www.programiz.com/kotlin-programming,Learn Kotlin Programming,About Kotlin Programming$$$Why Learn Kotlin?$$...,Kotlin is a relatively new programming languag...,,"Open Source - The Kotlin compiler, Intellij ID...",,
1,https://www.programiz.com/kotlin-programming/h...,Kotlin Hello World - You First Kotlin Program,"Kotlin ""Hello, World!"" Program$$$Comparison Wi...",,"In this article, you will learn to write Hello...","What is ""Hello, World!"" program?$$$Kotlin ""Hel...",// Hello World ProgramAny line starting with /...,
2,https://www.programiz.com/kotlin-programming/v...,Kotlin Variables and Basic Types,Kotlin Basic Types$$$Kotlin Arrays$$$Kotlin St...,,"In this tutorial, you will learn about variabl...",val (Immutable reference) - The variable decla...,,
3,https://www.programiz.com/kotlin-programming/o...,"Kotlin Operators: Arithmetic, Assignment Opera...",1. Arithmetic Operators$$$2. Assignment Operat...,,Kotlin has a set of operators to perform arith...,shl - Signed shift left$$$shr - Signed shift r...,,Operator|||Meaning$$$+|||Addition (also used f...
4,https://www.programiz.com/kotlin-programming/t...,Kotlin Type Conversion,Conversion from Larger to Smaller Type$$$Table...,,"In this article, you will learn about type con...",toByte() $$$toShort()$$$toInt()$$$toLong()$$$t...,,
5,https://www.programiz.com/kotlin-programming/s...,"Kotlin Expression, Statements and Blocks",Kotlin Expressions$$$Kotlin Statements$$$Kotli...,,"In this article, you will learn about Kotlin e...",Kotlin Expressions$$$Kotlin Statements$$$Kotli...,,
6,https://www.programiz.com/kotlin-programming/c...,Kotlin Comments,Traditional comment /* ... */$$$End of Line Co...,,"In this article, you will learn about Kotlin c...",/* ... */$$$// ....$$$What are comments in pro...,,
7,https://www.programiz.com/kotlin-programming/i...,Kotlin Basic Input/Output (With Examples),Koltin Output$$$Kotlin Input$$$Difference Betw...,,"In this article, you will learn to display out...",print() - prints string inside the quotes.$$$p...,,
8,https://www.programiz.com/kotlin-programming/i...,Kotlin if...else Expression (With Examples),Traditional Usage of if...else$$$Kotlin if exp...,,"In this article, you will learn to use if expr...",Traditional Usage of if...elseExample$$$Kotlin...,,
9,https://www.programiz.com/kotlin-programming/w...,Kotlin when Expression (With Examples),Kotlin when Construct$$$Few possibilities$$$Ex...,,"In this article, you will learn about when con...",Kotlin when Expression$$$Example: Simple Examp...,,


In [28]:
print(all_course_headings[7])
print('----------------------------------------------------------------------------------------')
dataset_7 = dataset_7.replace(np.nan, '', regex=True) # removing the NaN values from the dataset
dataset_7

Swift Programming
----------------------------------------------------------------------------------------


,Links,Main Heading,Chapter Headings,Description,Contents,Unordered Lists,Ordered Lists,Table Data
0,https://www.programiz.com/swift-programming,Learn Swift Programming,About Swift Programming$$$Why Learn Swift?$$$H...,Swift is a powerful and intuitive programming ...,,Open Source - Swift source code and developmen...,,
1,https://www.programiz.com/swift-programming/he...,"Swift ""Hello, World!"" Program","Swift ""Hello, World!"" Program$$$How Swift ""Hel...",,"In this tutorial, you will learn to write a pr...","Introduction$$$""Hello, World!"" Program$$$Working",,
2,https://www.programiz.com/swift-programming/va...,"Swift Variables, Constants and Literals (With ...",1. Swift Variables$$$Declare Variables in Swif...,,"In this tutorial, you will learn about variabl...",siteName is a variable of type String. Meaning...,Variables names must start with either a lette...,"Type|||Example|||Remarks$$$Decimal|||5, 10, -6..."
3,https://www.programiz.com/swift-programming/da...,Swift Data Types (With Examples),Swift Data Types$$$Swift Character$$$Swift Str...,,"In this tutorial, you will learn about data ty...",Size: Depends on the platform type$$$Range: -2...,,Data Types|||Example|||Description$$$Character...
4,https://www.programiz.com/swift-programming/ch...,Swift Characters and Strings (With Examples),Swift Character$$$Character Example$$$Swift St...,,"In this tutorial, you will learn about Swift c...","str1 and str2 are not equal. Hence, the result...",,Built-in function|||Description$$$isEmpty|||de...
5,https://www.programiz.com/swift-programming/ba...,Swift Basic Input and Output (With Examples),Swift Output$$$Example 1: Swift Print Statemen...,,"In this tutorial, we will learn simple ways to...",items - values inside the double quotation$$$s...,,
6,https://www.programiz.com/swift-programming/ex...,"Swift Expressions, Statements and Code blocks",Swift Expressions$$$Swift Statements$$$Swift C...,,"In this article, you will learn about Swift ex...",var marks = 80 - represents we are assigning 8...,,
7,https://www.programiz.com/swift-programming/co...,Swift Comments: How to use them and Why?,Single Line Comment$$$Multiline Comment$$$Use ...,,"In this article, you will learn about Swift co...",// - Single Line comments$$$/*...*/ - Multilin...,,
8,https://www.programiz.com/swift-programming/op...,Swift Optionals: How to Use them (With Examples),How to declare an Optional?$$$Optional Handlin...,,"In this article, you will learn about optional...",What is Swift Optional?$$$How to declare an op...,,
9,https://www.programiz.com/swift-programming/op...,Swift Operators (With Examples),Types of Operators$$$1. Swift Arithmetic Opera...,,"In this article, you'll learn everything about...",+ to add a and b$$$- to subtract b from a$$$* ...,Arithmetic operators$$$Assignment Operators$$$...,Operator|||Operation|||Example$$$+|||Addition|...


In [29]:
print(all_course_headings[8])
print('----------------------------------------------------------------------------------------')
dataset_8 = dataset_8.replace(np.nan, '', regex=True) # removing the NaN values from the dataset
dataset_8

C# Programming
----------------------------------------------------------------------------------------


,Links,Main Heading,Chapter Headings,Description,Contents,Unordered Lists,Ordered Lists,Table Data
0,https://www.programiz.com/csharp-programming,Learn C# Programming,About C# Programming$$$Why Learn C#?$$$How to ...,C# (pronounced as C sharp) is a general-purpos...,,Simple - The code written in C# is much simple...,,
1,https://www.programiz.com/csharp-programming/h...,C# Hello World - Your First C# Program,"""Hello World!"" in C#$$$How the ""Hello World!"" ...",,"In this tutorial, we will learn how to write a...",Every C# program must have a class definition....,// Hello World! Program// indicates the beginn...,
2,https://www.programiz.com/csharp-programming/k...,C# Keywords and Identifiers,C# Keywords$$$C# Identifiers$$$Rules for Namin...,,"In this tutorial, we will learn about keywords...",An identifier can not be a C# keyword.$$$An id...,,abstract|||as|||base|||bool$$$break|||byte|||c...
3,https://www.programiz.com/csharp-programming/v...,C# Variables and (Primitive) Data Types,How to declare variables in C#?$$$Rules for Na...,,"In this tutorial, we will learn about variable...",Boolean data type has two possible values: tru...,The variable name can contain letters (upperca...,Variable Names|||Remarks$$$name|||Valid$$$subj...
4,https://www.programiz.com/csharp-programming/o...,"C# Operators: Arithmetic, Comparison, Logical ...",1. Basic Assignment Operator$$$2. Arithmetic O...,,"In this article, we will learn everything abou...","If one of the operand is true, the OR operator...",,Operator|||Operator Name|||Example$$$+|||Addit...
5,https://www.programiz.com/csharp-programming/o...,C# Precedence And Associativity Of Operators: ...,C# Operator Precedence$$$Associativity of Oper...,,In this tutorial we you will learn about opera...,"At first, (--a) is evaluated resulting into 4....",,Category|||Operators$$$Postfix Increment and D...
6,https://www.programiz.com/csharp-programming/b...,"C# Bitwise Operators: AND, OR, XOR, Complement...",Bitwise OR$$$Bitwise AND$$$Bitwise XOR$$$Bitwi...,,"In this tutorial, we will learn in detail abou...",List of C# Bit Operators$$$Bitwise OR$$$Bitwis...,,Operator|||Operator Name$$$~|||Bitwise Complem...
7,https://www.programiz.com/csharp-programming/b...,C# Basic Input and Output,C# Output$$$C# Input$$$Example 1: Printing Str...,,"In this tutorial, we will learn how to take in...",ReadLine(): The ReadLine() method reads the ne...,,
8,https://www.programiz.com/csharp-programming/e...,"C# Expressions, Statements and Blocks (With Ex...",C# Expressions$$$C# Statements$$$C# Blocks$$$E...,,"In this article, we will learn about C# expres...","Selection Statements (if...else, switch)$$$Ite...",Declaration Statement$$$Expression Statement,
9,https://www.programiz.com/csharp-programming/c...,C# Comments: How to Use Them and Why?,Single Line Comments$$$Multi Line Comments$$$X...,,"In this article, we will learn about C# commen...",Instead comments should be used in the program...,Single Line Comments ( // )$$$Multi Line Comme...,
